calculate mult

In [1]:
loadlevel = 0.3

graphnamelist = ["dring","ls","rrg"]
swlist = range(40,121,20)
filenameprefix = "/home/annzhou/DRing/src/emp/datacentre/scalegraphfiles/"
flowfilenameprefix = "/home/annzhou/DRing/src/emp/datacentre/flowfiles/c2s_"
for sw in swlist:
    for graphname in graphnamelist:
        if graphname=="dring":
            filename = f"{filenameprefix}dring_deg{int(sw*0.8)}_sw{sw}_sn12_i1.edgelist"
        elif graphname=="rrg":
            filename = f"{filenameprefix}rrg_deg{int(sw*0.8)}_sw{sw}_svr{(sw//5)*(sw//5)*12}_os1_i1.edgelist"
        elif graphname=="ls":
            filename = f"{filenameprefix}ls_x{int(sw*0.6)}_y{sw//5}_i1.edgelist"
        with open(filename,'r') as f:
            numlink = len(f.readlines())
            
        flowfile = f"{flowfilenameprefix}{(sw//5)*(sw//5)*12}_{sw}_{int(sw*0.8)}_{int(sw*0.8)}_{int(sw*0.8)}_1_0_0_0"
        bytespermult = 0
        with open(flowfile,'r') as f:
            lines = f.readlines()
            for line in lines:
                tokens = line.split(',')
                bytespermult += int(tokens[2])
            
        bytesperlink = 894784*1500*0.2 # 894784 packets per second, 1500B per packet, 200ms
        totalbytes = bytesperlink*numlink # *2 for 2 directions, /2 for 2 hops per flow
        usablebytes = totalbytes*loadlevel
        mult = usablebytes/bytespermult
        print(f"{filename}: {mult}")

/home/annzhou/DRing/src/emp/datacentre/scalegraphfiles/dring_deg32_sw40_sn12_i1.edgelist: 1.0793276287547193
/home/annzhou/DRing/src/emp/datacentre/scalegraphfiles/ls_x24_y8_i1.edgelist: 1.0466207309136673
/home/annzhou/DRing/src/emp/datacentre/scalegraphfiles/rrg_deg32_sw40_svr768_os1_i1.edgelist: 1.0466207309136673
/home/annzhou/DRing/src/emp/datacentre/scalegraphfiles/dring_deg48_sw60_sn12_i1.edgelist: 0.4863209446807733
/home/annzhou/DRing/src/emp/datacentre/scalegraphfiles/ls_x36_y12_i1.edgelist: 0.4668681068935424
/home/annzhou/DRing/src/emp/datacentre/scalegraphfiles/rrg_deg48_sw60_svr1728_os1_i1.edgelist: 0.4668681068935424
/home/annzhou/DRing/src/emp/datacentre/scalegraphfiles/dring_deg64_sw80_sn12_i1.edgelist: 0.2715763054041317
/home/annzhou/DRing/src/emp/datacentre/scalegraphfiles/ls_x48_y16_i1.edgelist: 0.26136666986262297
/home/annzhou/DRing/src/emp/datacentre/scalegraphfiles/rrg_deg64_sw80_svr3072_os1_i1.edgelist: 0.26136666986262297
/home/annzhou/DRing/src/emp/datacentr

generate conffile

In [8]:
graphname = "dring"
routingname = "su2"
rt1 = "su"
rt2 = 2
swlist = range(40,121,20)
dringserverlist = [752,1680,2992,4672,6720]
multlistarr = [[1,5,100],[0,47,100],[0,26,100],[0,17,100],[0,12,100]]
sseedarr = range(1,2)

conffile = f"/home/annzhou/DRing/src/emp/datacentre/experiments/scalability/test.conf"
with open(conffile,'w') as f:
    for inumsw,numsw in enumerate(swlist):
        multlist = multlistarr[inumsw]
        multstr = f"{multlist[0]}_{multlist[1]}_{multlist[2]}"
        numlsserver = (numsw//5)*(numsw//5)*12
        numport = int(numsw*0.8) # == numleaf
        numdringserver = dringserverlist[inumsw]
        trafficfile = f"flowfiles/c2s_{numlsserver}_{numsw}_{numport}_{numport}_{numport}_{multstr}_0"
        lsqvarfile = f"qvarfiles/qvar_leafspine_{numlsserver}_{numsw}_{numport}_0_0_ecmp_64"
        lsnetpathfile = f"netpathfiles/netpath_ecmp_leafspine_{numlsserver}_{numsw}_{numport}"
        dringqvarfile = f"qvarfiles/qvar_{graphname}_{numdringserver}_{numsw}_{numport}_0_0_{routingname}_c2s_{numport}_{numport}_0_{multstr}_2_0_64"
        dringnetpathfile = f"netpathfiles/netpath_{routingname}_{graphname}_{numdringserver}_{numsw}_{numport}"
        dringscalegraphfile = f"scalegraphfiles/dring_deg{numport}_sw{numsw}_sn12_i1.edgelist"
        dringserverfile = f"serverfiles/dring_{numdringserver}_{numsw}_{numport}"
        for sseed in sseedarr:
            f.write(f"./run.sh LEAFSPINE MAKE {numsw} {numlsserver} {numport} leafspine_sw{numsw}_sseed{sseed} NEW_FILE null {multlist[0]} {multlist[1]} {multlist[2]} 0 0 0 0 0 0 ecmp 0 {trafficfile} null {sseed} {lsnetpathfile} {lsqvarfile} 64 50 150 200 0 0 > m_scalability_leafspine_{numsw}_{sseed}.log\n")
            f.write(f"./run.sh RRG MAKE {numsw} {numdringserver} {numport} dring_sw{numsw}_sseed{sseed} NEW_FILE {dringscalegraphfile} {multlist[0]} {multlist[1]} {multlist[2]} 0 0 0 0 0 0 {rt1} {rt2} {trafficfile} {dringserverfile} {sseed} {dringnetpathfile} {dringqvarfile} 64 50 150 200 0 0 > m_scalability_dring_{numsw}_{sseed}.log\n")

generate sumfile & picklefile

In [9]:
import pickle

homedir = "/home/annzhou"
datadict = dict()
conffile = f"{homedir}/DRing/src/emp/datacentre/experiments/scalability/test.conf"
sumfile = f"{homedir}/DRing/src/emp/datacentre/experiments/scalability/test.txt"
with open(conffile,'r') as f:
    with open(sumfile,'a') as sumf:
        lines = f.readlines()
        for line in lines:
            tokens = line.split()
            logfile = tokens[32]
            sizelist = list()
            durationlist = list()
            startlist = list()
            with open(f"{homedir}/DRing/src/emp/datacentre/{logfile}",'r') as logf:
                print(logfile)
                loglines = logf.readlines()
                for logline in loglines:
                    logtokens = logline.split()
                    if logtokens[0] == "FCT":
                        durationlist.append(float(logtokens[2]))
                        sizelist.append(int(logtokens[1]))
                        startlist.append(float(logtokens[3]))
            datadict[logfile] = [durationlist,sizelist,startlist]
            durationlist.sort()
            sumf.write(f"{logfile}\t{sum(durationlist)/len(durationlist)}\t{durationlist[int(len(durationlist)*0.5)]}\t{durationlist[int(len(durationlist)*0.99)]}\t{durationlist[int(len(durationlist)*0.9999)]}\n")

# with open(f'{homedir}/DRing/src/emp/datacentre/experiments/scalability/test.pickle', 'wb') as handle:
#     pickle.dump(datadict, handle, protocol=pickle.HIGHEST_PROTOCOL)

m_scalability_leafspine_40_1.log
m_scalability_dring_40_1.log
m_scalability_leafspine_60_1.log
m_scalability_dring_60_1.log
m_scalability_leafspine_80_1.log
m_scalability_dring_80_1.log
m_scalability_leafspine_100_1.log
m_scalability_dring_100_1.log
m_scalability_leafspine_120_1.log
m_scalability_dring_120_1.log


plot

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

trafficname = "prv1"

homedir = "/home/annzhou"
leafspinemultlistarr = [[0,2,100],[0,5,100],[0,7,100],[0,10,100],[0,12,100],[0,15,100],[0,17,100],[0,20,100]]
dringmultlistarr = [[0,2,100],[0,5,100],[0,7,100],[0,10,100],[0,12,100],[0,15,100],[0,17,100],[0,20,100],[0,22,100]]
leafspinemultnamearr = range(2,17,2)
dringmultnamearr = range(2,19,2)
fseedarr = range(5)
sumfile = f"{homedir}/DRing/src/emp/datacentre/experiments/routing/{trafficname}.txt"

xlist = list()
ylist_over_mult = list()
for imultlist,multlist in enumerate(leafspinemultlistarr):
    mult = multlist[0]
    numerator = multlist[1]
    denominator = multlist[2]
    multname = leafspinemultnamearr[imultlist]

    xlist.append(mult + numerator/denominator)

    with open(sumfile,'r') as f:
        lines = f.readlines()
        fct_over_fseed = list()
        for fseed in fseedarr:
            logfile = f"m_leafspine_{trafficname}_{multname}_{fseed}.log"
            for line in lines:
                tokens = line.split()
                if tokens[0] == logfile:
                    n99fct = float(tokens[3])
                    fct_over_fseed.append(n99fct)
        ylist_over_mult.append(sum(fct_over_fseed)/len(fct_over_fseed))
plt.plot(xlist,ylist_over_mult,label="leafspine",marker='x')

xlist = list()
ylist_over_mult = list()
for imultlist,multlist in enumerate(dringmultlistarr):
    mult = multlist[0]
    numerator = multlist[1]
    denominator = multlist[2]
    multname = dringmultnamearr[imultlist]

    xlist.append(mult + numerator/denominator)

    with open(sumfile,'r') as f:
        lines = f.readlines()
        fct_over_fseed = list()
        for fseed in fseedarr:
            logfile = f"m_dring_{trafficname}_{multname}_{fseed}.log"
            for line in lines:
                tokens = line.split()
                if tokens[0] == logfile:
                    n99fct = float(tokens[3])
                    fct_over_fseed.append(n99fct)
        ylist_over_mult.append(sum(fct_over_fseed)/len(fct_over_fseed))
plt.plot(xlist,ylist_over_mult,label="dring",marker='.')

plt.legend()
plt.xlabel('load (%)')
plt.ylabel('99ile fct (ms)')
plt.title(f'{trafficname}, 5 runs')
# plt.ylim(0,20)
plt.savefig(f"{homedir}/DRing/src/emp/datacentre/experiments/routing/{trafficname}.png", dpi=500)
plt.show()